In [12]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import pickle

In [14]:
import networkx as nx
from networkx.algorithms.cluster import clustering

In [20]:
import sys
sys.path.append('..')
from utils import gen_covariance

In [18]:
f = open('D:\\nse\\sim_final.dat', 'rb')

In [19]:
lasso = pickle.load(f)

In [21]:
cov = gen_covariance(1000, **lasso.iloc[0]['cov_params'])

In [22]:
G = nx.from_numpy_matrix(cov)

In [25]:
%time clustering(G, weight = 'weight')

Wall time: 25min 22s


{0: 0.03392345592313427,
 1: 0.03412731830616714,
 2: 0.03433115993943338,
 3: 0.034534973332939936,
 4: 0.03473875110268355,
 5: 0.03494248596956355,
 6: 0.03514617075830462,
 7: 0.03534979839638977,
 8: 0.03555336191300405,
 9: 0.035756854437986756,
 10: 0.03596026920079513,
 11: 0.03616359952947673,
 12: 0.036366838849652465,
 13: 0.036569980683507745,
 14: 0.036773018648795064,
 15: 0.03697594645784429,
 16: 0.03717875791658352,
 17: 0.037381446923568526,
 18: 0.03758400746902142,
 19: 0.03778643363387891,
 20: 0.037988719588848516,
 21: 0.038190859593475214,
 22: 0.03839284799521516,
 23: 0.03859467922852004,
 24: 0.03879634781392812,
 25: 0.03899784835716533,
 26: 0.03919917554825441,
 27: 0.03940032416063225,
 28: 0.03960128905027558,
 29: 0.03980206515483533,
 30: 0.04000264749277915,
 31: 0.04020303116254113,
 32: 0.04040321134168077,
 33: 0.04060318328604902,
 34: 0.04080294232896236,
 35: 0.041002483880385375,
 36: 0.04120180342611958,
 37: 0.04140089652700211,
 38: 0.041599

In [33]:
from networkx.algorithms.assortativity.correlation import degree_pearson_correlation_coefficient

In [ ]:
%time degree_pearson_correlation_coefficient(G, weight = 'weight')